In [26]:
using NamedColors
using Colors
using ColorTypes
using StatsBase
using Learn
using MLDataUtils
using Iterators
using MultivariateStats

INFO: Precompiling module MultivariateStats.


In [28]:
const allowed_chars = Set(['a':'z'; '\''; ' '; '/'])

Set(['d','q',' ','/','e','v','u','z','c','j'  …  's','n','f','i','y','x','r','o','w','k'])

In [29]:
mega_dict = merge(load_crayola(),
                      load_resene(),
                      load_nbs(),
                      load_xkcd(),
                      load_x11())
test_training_dict=similar(mega_dict)
for (kk, vv) in mega_dict
    name = lowercase(kk)
    length(setdiff( Set(name), allowed_chars)) > 0  && continue
    test_training_dict[name] = vv
end
test_training_dict

Dict{String,ColorTypes.RGB24} with 2721 entries:
  "maximum red"        => RGB24{U8}(0.851,0.129,0.129)
  "grass green"        => RGB24{U8}(0.247,0.608,0.043)
  "celery"             => RGB24{U8}(0.757,0.992,0.584)
  "lipstick"           => RGB24{U8}(0.671,0.02,0.388)
  "japanese maple"     => RGB24{U8}(0.471,0.004,0.035)
  "darkorchid"         => RGB24{U8}(0.6,0.196,0.8)
  "au chico"           => RGB24{U8}(0.592,0.376,0.365)
  "laurel"             => RGB24{U8}(0.455,0.576,0.471)
  "chocolate"          => RGB24{U8}(0.216,0.008,0.008)
  "loulou"             => RGB24{U8}(0.275,0.043,0.255)
  "fuzzy wuzzy"        => RGB24{U8}(0.529,0.259,0.122)
  "astronaut"          => RGB24{U8}(0.157,0.227,0.467)
  "ice cold"           => RGB24{U8}(0.694,0.957,0.906)
  "floral white"       => RGB24{U8}(1.0,0.98,0.941)
  "pesto"              => RGB24{U8}(0.486,0.463,0.192)
  "purplishblack"      => RGB24{U8}(0.141,0.129,0.141)
  "pine green"         => RGB24{U8}(0.039,0.282,0.118)
  "judge grey"         =

In [30]:
function encode_output(raw::Colorant)
    #Encoding Angles: http://stats.stackexchange.com/a/218547/36769
    hsv = convert(HSV, raw)
    [(sind(hsv.h)+1)/2, (cosd(hsv.h)+1)/2, hsv.s, hsv.v]
end

function decode_output(out)
    hue = rad2deg(atan2(2out[1]-1, 2out[2]-1))
    sat = out[3]
    val = out[4]
    HSV(hue, sat, val)
end


decode_output (generic function with 1 method)

In [31]:
const nchars = 26+2
function char2index(c::Char)::Int
    if c ∈ 'a':'z'
        c-'a'
    elseif c==' '
        'z'+1
    elseif c=='/'
        'z'+2
    elseif c=='\''
        'z'+3
    else
        error("$c is out of range")
    end
end

"""
Builds a ngram_encoder, out of a alphabet encoder
`n` the length of the ngrams
`m` the length of the alphabet
`alpha_encoder` must map the alphabet to the range 0:alphasize-1
"""
function ngram_encoder(alpha_encoder, alphasize)
    function ngram_encoder_inner(alphastring)
        n=length(alphastring)
        n_ngrams = alphasize^n-1
        alphasize.^(0:n-1)⋅alpha_encoder.(collect(alphastring))
    end
end

function encode_input(name::AbstractString; n=2)
    alphasize = 26+3
    coder = ngram_encoder(char2index, alphasize)
    
    ngrams = partition(name, n, 1) |> collect
    name_inds::Vector{Int} = coder.(ngrams)
    counts(name_inds, alphasize^n)
end

encode_input (generic function with 1 method)

In [42]:
function get_data(data_dict)

    colors = collect(values(data_dict))
    ys = hcat(encode_output.(colors)...)
    
    names = collect(keys(data_dict))
    xs = hcat(encode_input.(names)...).*1.0

    #These could all be views, but PCA don't like that
    x_train = xs[:, 1:9*end÷10]
    y_train = ys[:, 1:9*end÷10]
    x_test = xs[:,1+9*end÷10 : end]
    y_test = ys[:,1+9*end÷10 : end]
    
    (x_train, y_train), (x_test, y_test)
end


get_data (generic function with 1 method)

In [57]:
function full_encode_input(M::PCA, name::AbstractString)
    x=encode_input(name)*1.0
    MultivariateStats.transform(M,x)
end

full_encode_input (generic function with 1 method)

In [43]:
(x_train, y_train), (x_test, y_test) = get_data(test_training_dict)
M = fit(PCA, collect(x_train))#; maxoutdim=100)



x_train_dr = MultivariateStats.transform(M,x_train)
x_test_dr =  MultivariateStats.transform(M,x_test)
(train,test) = (x_train_dr, y_train), (x_test_dr, y_test)

((
[-0.113588 -2.22179 … 0.425229 0.443863; -0.331308 -0.495611 … -0.318987 -0.110415; … ; -0.000409796 0.00596641 … -0.0509405 -0.040835; -0.00746094 0.000719227 … 0.0742557 0.0219216],

Float32[0.5 0.994721 … 0.957656 0.799331; 1.0 0.427534 … 0.701373 0.900501; 0.847926 0.929032 … 0.197531 0.925373; 0.85098 0.607843 … 0.952941 0.52549]),(
[0.433017 -1.35889 … 0.493951 0.458666; -0.385176 -0.154454 … -0.381923 -0.462611; … ; 0.13126 0.112774 … -0.0171019 -0.0109848; -0.0510445 0.0177422 … 0.0193203 -0.0425417],

Float32[0.75 0.252875 … 0.691342 0.0173382; 0.933013 0.0653399 … 0.96194 0.630528; 0.121212 0.963415 … 0.125 0.79021; 0.647059 0.321569 … 0.25098 0.560784]))

In [45]:
function my_test_loss(obj, testdata)
    totloss = 0.0
    totcorrect = 0
    for (x,y) in eachobs(testdata)
        totloss += transform!(obj,y,x)
    end
    totloss#, totcorrect/totcount
end


tracer = IterFunction() do obj, i
    if mod1(i,1000)==1000
        totloss  = my_test_loss(obj, test)
        println(i, " totloss:  ", totloss)
        #println(i, " accuracy: ", accuracy)
    end
end


StochasticOptimization.IterFunction(#13)

In [68]:
nin, nh, nout = size(train[1],1), [100,50], size(train[2],1)
t = nnet(nin, nout, nh, :softplus, :logistic)
obj = objective(t, ElasticNetPenalty(1e-5))

ObjectiveFunctions.RegularizedObjective{Transformations.Chain{Float64,Transformations.Params{SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{},Tuple{}}},ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss},Penalties.ElasticNetPenalty{Float64}}(Chain{Float64}(
   Affine{308-->100}
   softplus{100}
   Affine{100-->50}
   softplus{50}
   Affine{50-->4}
   logistic{4}
) ,ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss}(LossFunctions.CrossentropyLoss(),4,input(4,),target(4,),output(1,)),Penalties.ElasticNetPenalty{Float64}(1.0e-5,0.5))

In [ ]:
learner = make_learner(
    GradientLearner(5e-2, Adadelta()),
    tracer,
    maxiter = 50_000
)
learn!(obj, learner, infinite_batches(train, size=5))

1000 totloss:  689.3251360301238
2000 totloss:  662.421913843605
3000 totloss:  651.7005110807384
4000 totloss:  662.0646111298684
5000 totloss:  657.526558910904
6000 totloss:  649.1826726229634
7000 totloss:  655.9272274976371
8000 totloss:  656.579723838186
9000 totloss:  655.2514199081426
10000 totloss:  653.0486983217468
11000 totloss:  655.7827835012451
12000 totloss:  659.5419084347171
13000 totloss:  657.1112466061549
14000 totloss:  656.4801992098115
15000 totloss:  662.6702967495486
16000 totloss:  660.5369423705812
17000 totloss:  657.3447676693083
18000 totloss:  672.1715732820178
19000 totloss:  658.7626628447936
20000 totloss:  666.3839124329943
21000 totloss:  659.2240428821248
22000 totloss:  662.9122089250316
23000 totloss:  658.2127588732945
24000 totloss:  661.214003812204
25000 totloss:  658.3234921753344
26000 totloss:  662.1875340779209
27000 totloss:  663.9164988648754
28000 totloss:  669.0530400126092
29000 totloss:  668.2882571218045
30000 totloss:  670.2915027

In [ ]:
full_encode_input(M,"hyperblue")

In [70]:
function get_colour(obj, M, name)
    x = full_encode_input(M,name)
    decode_output( output_value(obj.transformation ))
end
get_colour(name) = get_colour(obj, M, name)


function display_colour(obj, M,name)
    display(name)
    display(get_colour(obj, M, name))
    display("-"^16)
end
display_colour(name)=display_colour(obj, M,name)

display_colour (generic function with 3 methods)

In [72]:
display_colour("green")
display_colour("red")
display_colour("lightred")
display_colour("dirtyred")
display_colour("lightredishblue")
display_colour("bluish red")
display_colour("blue")
display_colour("light blue")
display_colour("lightred")
 


"green"

"----------------"

"red"

"----------------"

"lightred"

...

"----------------"

"light blue"

"----------------"

"lightred"

"----------------"

In [73]:
display_colour("hot pink")
display_colour("hot blue")
display_colour("hot green")
display_colour("dark blue")
display_colour("light black")
display_colour("black")
display_colour("dark black")

"hot pink"

"----------------"

"hot blue"

"----------------"

"hot green"

...

"----------------"

"black"

"----------------"

"dark black"

"----------------"

In [ ]:
for letter in ['a':'z'; ' '; '\''; '/']
    display_colour(obj, string(letter))
end

In [ ]:
for letter1 in ['a':'z'; ' '; '\''; '/']
    for letter2 in ['a':'z'; ' '; '\''; '/']
        display_colour(obj, join([letter1,letter2]))
    end
end

In [ ]:
1+1